<h2>Whole-slide inference using histomics-stream </h2>

This example notebook demonstrates how to use Histomics Stream to perform detection inference on a whole-slide image. Histomics Stream prefetches and queues tiles for inference, and can be used with the predict function of a keras detection model to do serial or parallel inference.

In [ ]:
# install histomics_detect
!pip install -e /tf/notebooks/histomics_detect

# install histomics_stream
!pip install -e /tf/notebooks/histomics_stream

# add to system path
import sys

sys.path.append("/tf/notebooks/histomics_detect/")
sys.path.append("/tf/notebooks/histomics_stream/")

<h2>Download slide and trained model</h2>

This example uses a model trained for nuclei detection in breast cancer (see basic example notebook). To illustrate whole-slide inference we apply this model to a 40X objective magnification image from The Cancer Genome Atlas. We also use a mask of the foreground tissue region to avoid uncessesary inference on background tiles.

In [ ]:
# import dataset related packages
from histomics_detect.models import FasterRCNN
import os
import pooch
import tensorflow as tf

# download whole slide image
wsi_path = pooch.retrieve(
    fname='TCGA-AN-A0G0-01Z-00-DX1.svs',
    url='https://northwestern.box.com/shared/static/qelyzb45bigg6sqyumtj8kt2vwxztpzm',
    known_hash='d046f952759ff6987374786768fc588740eef1e54e4e295a684f3bd356c8528f',
    path=str(pooch.os_cache('pooch')) + os.sep + 'wsi'
)

# download binary mask image
mask_path = pooch.retrieve(
    fname='TCGA-AN-A0G0-01Z-00-DX1.mask.png',
    url='https://northwestern.box.com/shared/static/2q13q2r83avqjz9glrpt3s3nop6uhi2i',
    known_hash='bb657ead9fd3b8284db6ecc1ca8a1efa57a0e9fd73d2ea63ce6053fbd3d65171',
    path=str(pooch.os_cache('pooch')) + os.sep + 'wsi'
)

# download trained model
model_path = pooch.retrieve(
    fname='tcga_brca_model',
    url='https://northwestern.box.com/shared/static/wharwfevgyl6qe60hgwchdml67gucq89',
    known_hash='410bc16985d1959472d04c6250b4f6fccf60c5ddd31c910254a0f0a605746132',
    path=str(pooch.os_cache('pooch')) + os.sep + 'model',
    processor=pooch.Unzip()
)
model_path = os.path.split(model_path[0])[0]

# restore keras model
model = tf.keras.models.load_model(model_path, custom_objects={'FasterRCNN': FasterRCNN})

<h2>Build Dataset from dictionary of instructions</h2>

We use Histomics Stream to create a tf.data.Dataset of tiles for feeding the detection model. This allows prefetching and queueing of tiles directly from the whole-slide image and simplifies multi-GPU inference.

In [ ]:
import histomics_stream as hs

# define analysis parameters for a single slide
slide = {'filename':  wsi_path,
         'slide_name': 'TCGA-AN-A0G0-01Z-00-DX1', #slide name without extention
         'slide_group': 'TCGA-AN-A0G0', #used to group multiple slides
         'number_pixel_rows_for_chunk': 2048,
         'number_pixel_columns_for_chunk': 2048}

# add slide to study
study = {'version': 'version-1',
         'number_pixel_rows_for_tile': 1024,
         'number_pixel_columns_for_tile': 1024,
         'slides': {'TCGA-AN-A0G0-01Z-00-DX1': slide}}

# find the best resolution for each slide given the desired_magnification
resolution = hs.configure.FindResolutionForSlide(study, desired_magnification=40, magnification_tolerance=0.02)
for slide in study["slides"].values():
    resolution(slide)

# define grid by adding mask and defining tile overlap 
grid_and_mask = hs.configure.TilesByGridAndMask(
    study,
    number_pixel_overlap_rows_for_tile=32,
    number_pixel_overlap_columns_for_tile=32,
    mask_filename=mask_path
)
    
# apply to all slides - we have 1 slide but show this for 
for slide in study["slides"].values():
    grid_and_mask(slide)
        
# create dataset
ds = hs.tensorflow.CreateTensorFlowDataset()
tiles = ds(study)

<h2>Using .predict with a model wrapper</h2>

.predict() is the most efficient method for inference, however, it combines results from all tiles into a single array. Since these results are in local tile coordinates, the global position of the tile within the slide needs to be included to locale the nuclei within the slide. To do this we create a simple model wrapper that adds the global tile positions to the tile-based inference results.

In [ ]:
import time

# define the wrapped model class
class WrappedModel(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super(WrappedModel, self).__init__(*args, **kwargs)
        self.model = model

    def call(self, inputs, *args, **kwargs):
        boxes = self.model(inputs[0], *args, **kwargs)
        x = tf.cast(inputs[1]['tile_left'], tf.float32) * tf.ones((tf.shape(boxes)[0], 1))
        y = tf.cast(inputs[1]['tile_top'], tf.float32) * tf.ones((tf.shape(boxes)[0], 1))    
        return (tf.concat([boxes, x, y], 1), inputs[1])

# wrap the model
wrapped_model = WrappedModel(model, name='wrapped_model')

# .predict inference
start = time.time()
inference = wrapped_model.predict(tiles)
print('%d nuclei in %d tiles: %.2f seconds' % (inference[0].shape[0],
                                               len(inference[1]['tile_top']),
                                               time.time()-start))

<h3>Parallel inference with .predict</h3>

In [ ]:
# create a strategy that mirrors the model across GPUs
strategy = tf.distribute.MirroredStrategy()

# reformat tiled dataset to replace 'None' values with dummy zeros
padded = tiles.map(lambda x, y, z: (x, 0., 0.))

# change wrapper class to handle batch dimension, empty batches, and to capture tile location
class WrappedModel(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super(WrappedModel, self).__init__(*args, **kwargs)
        self.model = model

    def call(self, inputs, *args, **kwargs):
        boxes = tf.cond(tf.greater(tf.size(inputs[0]), 0),
                        lambda: self.model(inputs[0][0,:,:,:], *args, **kwargs),
                        lambda: tf.zeros((0,4)))
        x = tf.cast(inputs[1]['tile_left'], tf.float32) * tf.ones((tf.shape(boxes)[0], 1))
        y = tf.cast(inputs[1]['tile_top'], tf.float32) * tf.ones((tf.shape(boxes)[0], 1))    
        return (tf.concat([boxes, x, y], 1), inputs[1])

# restore and wrap the model in a distributed strategy context
with strategy.scope():
    
    # restore keras model in distributed strategy
    model = tf.keras.models.load_model(model_path, custom_objects={'FasterRCNN': FasterRCNN})

    # wrap the model
    wrapped = WrappedModel(model, name='wrapped_model')
    
# batch to number of GPUs in strategy
batched = padded.batch(strategy.num_replicas_in_sync)

# .predict inference
start = time.time()
inference = wrapped.predict(batched)
print('%d nuclei in %d tiles: %.2f seconds on %d GPUs' % (inference[0].shape[0],
                                                          len(inference[1]['tile_top']),
                                                          time.time()-start,
                                                          strategy.num_replicas_in_sync))

<h3>Comparison to list comprehension with tf.data.Dataset.map()</h3>

Note: this can be extremely slow.

In [ ]:
from histomics_detect.visualization import plot_inference

dataset_map_options = {
    "num_parallel_calls": tf.data.experimental.AUTOTUNE,
    "deterministic": False,
}

# define a dataset containing the input image, the tile inference results, and tile metadata
tiles = tiles.map(lambda x, y, z: (x[0], model(x[0], tau=0.5, nms_iou=0.2, margin=32), x[1]), **dataset_map_options)

# pull the tiles from the dataset using list comprehension
start = time.time()
results = [result for result in tiles]
print('list comprehension: %.2f' % (time.time()-start))

# visualize the tile with the largest number of detections
detections = [result[1].shape[0] for result in results]
index = detections.index(max(detections))
plot_inference(results[index][0], results[index][1])